<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# Eikon Data API

**Python Wrapper Package &mdash; Data Retrieval**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial covers the **basic functions** of the `eikon` Python wrapper package:

* `eikon` package
* `ek.get_symbology()`
* `ek.get_data()`
* `ek.get_timeseries()`
* `ek.get_news_headlines()`
* `ek.get_news_story()`

## Installation

The **Python wrapper package** for the Thomson Reuters Eikon Data API is called `eikon`.

It is **installed** via `pip`:
    
    pip install eikon
    
It can also be **upgraded** via `pip`:

    pip install eikon --upgrade

## Imports and Versions

The following imports several **packages** as used in the following.

In [1]:
import eikon as ek  # the Eikon Python wrapper package
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [2]:
import sys
print(sys.version)

3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [3]:
ek.__version__

'1.1.2'

In [4]:
cf.__version__

'0.17.3'

## Connecting to Eikon Data API

This code sets the `app_id` to connect to the **Eikon Data API Proxy** which needs to be running locally. It requires the previously created text file `eikon.cfg` to be in the current working directory.

In [5]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location

[]

In [7]:
import eikon as ek
ek.set_app_key('0ed6a35e0937415eab446d3375bca7cf671d6b4c')

## Functions

The Python function `help()` can be used to look up the **help texts** for all elements of the Python wrapper package.

In [8]:
help(ek)

Help on package eikon:

NAME
    eikon - # coding: utf-8

PACKAGE CONTENTS
    Profile
    data_grid
    eikonError
    json_requests
    news_request
    streaming_session (package)
    symbology
    time_series
    tools

SUBMODULES
    cache
    desktop_session
    istream_callback
    itemstream
    session
    stream
    stream_connection
    streamingprice
    streamingprice_callback
    streamingprices

VERSION
    1.1.2

FILE
    c:\anacondaenvs\envs\py37\lib\site-packages\eikon\__init__.py




In addition, from within Jupyter Notebook, for example, you can use **tab completion** to look up available functionality:

In [ ]:
# ek.  # uncomment & place cursor behind dot, then press the tab key

What you see should look similar to this screenshot:

<img src="eikon_tab_completion.png" width="45%" align="left">

The `eikon` package provides the following **basic functions** to interact with the Eikon Data API and to retrieve data of different types:

* `ek.get_symbology()`: converting symbology types
* `ek.get_data()`: retrieve historical data
* `ek.get_timeseries()`: retrieve historical time series data
* `ek.get_news_headlines()`: retrieve news headlines
* `ek.get_news_story()`: retrieve full news texts

## `ek.get_symbology()`

The function `ek.get_symbology()` allows you to **convert symbols** from one symbology type to another.

In [9]:
help(ek.get_symbology)

Help on function get_symbology in module eikon.symbology:

get_symbology(symbol, from_symbol_type='RIC', to_symbol_type=None, raw_output=False, debug=False, bestMatch=True)
    Returns a list of instrument names converted into another instrument code.
    For example: convert SEDOL instrument names to RIC names
    
    Parameters
    ----------
    symbol: string or list of strings
        Single instrument or list of instruments to convert.
    
    from_symbol_type: string
        Instrument code to convert from.
        Possible values: 'CUSIP', 'ISIN', 'SEDOL', 'RIC', 'ticker', 'lipperID', 'IMO'
        Default: 'RIC'
    
    to_symbol_type: string or list
        Instrument code to convert to.
        Possible values: 'CUSIP', 'ISIN', 'SEDOL', 'RIC', 'ticker', 'lipperID', 'IMO', 'OAPermID'
        Default: None  (means all symbol types are requested)
    
    raw_output: boolean, optional
        Set this parameter to True to get the data in json format
        if set to False, 

Assume the following set of `RICs`.

In [11]:
rics = [
    'AAPL.O',  # Apple stock
    'AMZN.O',  # Amazon stock
    'TSLA.O',  # Tesla stock
    '.SPX',  # S&P 500 stock index
]

The **conversion** works as follows:

In [12]:
isin = ek.get_symbology(rics, from_symbol_type='RIC', to_symbol_type='ISIN')

In [13]:
isin

,ISIN
AAPL.O,US0378331005
AMZN.O,US0231351067
TSLA.O,US88160R1014
.SPX,US78378X1072


And also the other way round, after a conversion of data structures:

In [14]:
isins = list(isin['ISIN'].values)

In [15]:
isins

['US0378331005', 'US0231351067', 'US88160R1014', 'US78378X1072']

In [16]:
ek.get_symbology(isins, from_symbol_type='ISIN', to_symbol_type='RIC')

,RIC
US0378331005,AAPL.O
US0231351067,AMZN.O
US88160R1014,TSLA.O
US78378X1072,.SPX


## `ek.get_data()`

The function `ek.get_data()` is a **general purpose data retrieval** function.

In [17]:
help(ek.get_data)

Help on function get_data in module eikon.data_grid:

get_data(instruments, fields, parameters=None, field_name=False, raw_output=False, debug=False)
    Returns a pandas.DataFrame with fields in columns and instruments as row index
    
    Parameters
    ----------
    instruments: string or list
        Single instrument or list of instruments to request.
    
    fields: string, dictionary or list of strings and/or dictionaries.
        List of fields to request.
    
        Examples:
    
        - 'TR.PriceClose'
        - {'TR.GrossProfit': { 'params':{ 'Scale': 6, 'Curn': 'EUR' }}
        - {'TR.GrossProfit': { 'params':{ 'Scale': 6, 'Curn': 'EUR' },sort_dir:'desc'}
        - ['TR.PriceClose','TR.PriceOpen']
        - [{'TR.PriceClose':  {'sort_dir':asc,sort_priority:1}},{'TR.PriceOpen':  {'sort_dir':asc,sort_priority:0}}
        
        You can use the function TR_Field to build the fields:
    
        >>> fields = [ek.TR_Field('tr.revenue'),ek.TR_Field('tr.open','asc',1),e

In [18]:
data_grid, err = ek.get_data(['IBM', 'GOOG.O', 'MSFT.O'],
                             ['TR.TotalReturnYTD', 'TR.WACCBeta', 'YRHIGH', 'YRLOW'])

In [19]:
data_grid

,Instrument,YTD Total Return,Beta,YRHIGH,YRLOW
0,IBM,-5.970579,1.289669,158.7500,90.5600
1,GOOG.O,-4.316315,1.034526,1532.1063,1013.5361
2,MSFT.O,10.986261,0.950900,190.7000,119.0100


In [20]:
data_grid.set_index('Instrument')[['YTD Total Return', 'Beta']].iplot(
                    kind='bar', subplots=True)

In [21]:
data_grid.set_index('Instrument')[['YRHIGH', 'YRLOW']].iplot(kind='bar')

## `ek.get_timeseries()`

The function `ek.get_timeseries()` retrieves **time series data**.

In [22]:
help(ek.get_timeseries)

Help on function get_timeseries in module eikon.time_series:

get_timeseries(rics, fields='*', start_date=None, end_date=None, interval='daily', count=None, calendar=None, corax=None, normalize=False, raw_output=False, debug=False)
    Returns historical data on one or several RICs
    
    Parameters
    ----------
    rics: string or list of strings
        Single RIC or List of RICs to retrieve historical data for
    
    start_date: string or datetime.datetime or datetime.timedelta
        Starting date and time of the historical range.
        string format is: '%Y-%m-%dT%H:%M:%S'. e.g. '2016-01-20T15:04:05'.
        datetime.timedelta is negative number of day relative to datetime.now().
        Default: datetime.now() + timedelta(-100)
        You can use the helper function get_date_from_today, please see the usage in the examples section
    
    end_date: string or datetime.datetime or datetime.timedelta
        End date and time of the historical range.
    
        string 

For instance, the following code retries **daily closing prices**  for the four `RICs` as defined above.

In [24]:
rics

['AAPL.O', 'AMZN.O', 'TSLA.O', '.SPX']

In [23]:
data = ek.get_timeseries(rics,
                         start_date='2017-01-01',
                         end_date='2018-02-23',
                         fields='CLOSE',
                         interval='daily'
                        )

In [25]:
data.head()

CLOSE,AAPL.O,AMZN.O,TSLA.O,.SPX
Date,,,,
2017-01-03,116.15,753.67,216.99,2257.83
2017-01-04,116.02,757.18,226.99,2270.75
2017-01-05,116.61,780.45,226.75,2269.00
2017-01-06,117.91,795.99,229.01,2276.98
2017-01-09,118.99,796.92,231.28,2268.90


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 288 entries, 2017-01-03 to 2018-02-23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL.O  288 non-null    float64
 1   AMZN.O  288 non-null    float64
 2   TSLA.O  288 non-null    float64
 3   .SPX    288 non-null    float64
dtypes: float64(4)
memory usage: 11.2 KB


Making use of `Cufflinks`, the data is visualized below.

In [27]:
data.normalize().iplot()

## `ek.get_news_headlines()`

The function `ek.get_news_headlines()` allows you to search for and retrieve **news headlines**, including `storyId` values needed to retrieve the full news text.

In [28]:
help(ek.get_news_headlines)

Help on function get_news_headlines in module eikon.news_request:

get_news_headlines(query='Topic:TOPALL and Language:LEN', count=10, date_from=None, date_to=None, raw_output=False, debug=False)
    Returns a list of news headlines
    
    Parameters
    ----------
    query: string, optional
        News headlines search criteria.
        The text can contain RIC codes, company names, country names and
        operators (AND, OR, NOT, IN, parentheses and quotes for explicit search…).
    
        Tip: Append 'R:' in front of RIC names to improve performance.
    
        Default: Top News written in English
    
    count: int, optional
        Max number of headlines retrieved.
    
        Value Range: [1-100].
    
        Default: 10
    
    date_from: string or datetime, optional
        Beginning of date range.
    
        String format is: '%Y-%m-%dT%H:%M:%S'. e.g. '2016-01-20T15:04:05'.
    
    date_to: string or datetime, optional
        End of date range.
    
        

A `query` string might contain `RICs` and other words to be searched for.

In [29]:
rics[2]

'TSLA.O'

In [39]:
news = ek.get_news_headlines('R:%s PRODUCTION IN ENGLISH' % rics[2],
                         date_from='2016-02-01',
                         date_to='2020-02-14',
                         count=100
                        )

In [40]:
'R:%s PRODUCTION IN ENGLISH' % rics[2]

'R:TSLA.O PRODUCTION IN ENGLISH'

In [41]:
news

,versionCreated,text,storyId,sourceCode
2020-02-11 09:00:13.155,2020-02-11 09:00:13.155000+00:00,Tesla Stock Climbs As Its Shanghai Plant Resta...,urn:newsml:reuters.com:20200211:nCXG21B1xk:1,NS:CAIXIN
2020-02-10 12:54:56.693,2020-02-10 14:53:27.844000+00:00,Reuters Insider - Tesla shares pop in pre-mark...,urn:newsml:reuters.com:20200210:nRTV36p28H:6,NS:CNBC
2020-02-10 12:46:30.000,2020-02-10 12:46:30+00:00,BUZZ-Tesla Inc: Rallies as production restarts...,urn:newsml:reuters.com:20200210:nL4N2AA2YM:1,NS:RTRS
2020-02-10 11:34:21.418,2020-02-10 11:34:21.418000+00:00,TESLA SHARES UP 7.8% PREMARKET; CO'S FACTORY I...,urn:newsml:reuters.com:20200210:nFWN2AA121:1,NS:RTRS
2020-02-08 04:38:20.000,2020-02-08 10:15:40+00:00,UPDATE 2-Shanghai to help Tesla resume product...,urn:newsml:reuters.com:20200208:nL3N2A8019:3,NS:RTRS
2020-02-08 03:22:34.000,2020-02-08 03:45:57+00:00,Tesla Shanghai factory to resume production on...,urn:newsml:reuters.com:20200208:nB9N28F04L:1,NS:RTRS
2020-02-08 03:22:33.654,2020-02-08 03:22:33.654000+00:00,SHANGHAI GOVT OFFICIAL SAYS TESLA FACTORY WILL...,urn:newsml:reuters.com:20200208:nB9N28F04L:1,NS:RTRS
2020-02-07 09:30:48.491,2020-02-07 09:30:48.491000+00:00,LG CHEM SAYS IT PLANS TO PARTIALLY RESUME PROD...,urn:newsml:reuters.com:20200207:nS6N29P03J:1,NS:RTRS
2020-02-05 13:00:16.097,2020-02-05 13:00:16.097000+00:00,Tesla’s Shanghai Factory To Start Production B...,urn:newsml:reuters.com:20200205:nCXG6RZ151:1,NS:CAIXIN
2020-02-05 09:18:17.964,2020-02-05 09:19:03.974000+00:00,TESLA INC SENIOR EXEC SAYS SHANGHAI PLANT PLAN...,urn:newsml:reuters.com:20200205:nB9N28F04E:3,NS:RTRS


## `ek.get_news_story()`

The function `ek.get_news_story()` retrieves the full text of a **news stroy** given the `storyId` value.

In [42]:
help(ek.get_news_story)

Help on function get_news_story in module eikon.news_request:

get_news_story(story_id, raw_output=False, debug=False)
    Return a single news story corresponding to the identifier provided in story_id
    
    Parameters
    ----------
    story_id: string
        The story id is a field you will find in every headline you retrieved with the function get_news_headlines.
    
    raw_output: boolean
        Set this parameter to True to get the data in json format
        if set to False, the function will return returns the story content.
    
        The default value is False.
    
    debug: boolean, optional
        When set to True, the json request and response are printed.
        Default: False
    
    Raises
    ------
        Exception
            If http request fails or if Refinitiv Services return an error.
    
        ValueError
            If a parameter type or value is wrong.
    
    Examples
    --------
    >>> import eikon as ek
    >>> ek.set_app_key('set your

The `storyId` values are stored in the respective column of the `news` `DataFrame` object as created above.

In [43]:
news['storyId']

2020-02-11 09:00:13.155    urn:newsml:reuters.com:20200211:nCXG21B1xk:1
2020-02-10 12:54:56.693    urn:newsml:reuters.com:20200210:nRTV36p28H:6
2020-02-10 12:46:30.000    urn:newsml:reuters.com:20200210:nL4N2AA2YM:1
2020-02-10 11:34:21.418    urn:newsml:reuters.com:20200210:nFWN2AA121:1
2020-02-08 04:38:20.000    urn:newsml:reuters.com:20200208:nL3N2A8019:3
2020-02-08 03:22:34.000    urn:newsml:reuters.com:20200208:nB9N28F04L:1
2020-02-08 03:22:33.654    urn:newsml:reuters.com:20200208:nB9N28F04L:1
2020-02-07 09:30:48.491    urn:newsml:reuters.com:20200207:nS6N29P03J:1
2020-02-05 13:00:16.097    urn:newsml:reuters.com:20200205:nCXG6RZ151:1
2020-02-05 09:18:17.964    urn:newsml:reuters.com:20200205:nB9N28F04E:3
Name: storyId, dtype: object

To read a news story, **pick out one story via its storyId and display the full text** provided as HTML code.

In [45]:
storyId = news['storyId'][0]
storyId

'urn:newsml:reuters.com:20200211:nCXG21B1xk:1'

In [48]:
from IPython.display import display,HTML
display(HTML('<h1>Hello,World</h1>'))

In [47]:
HTML(ek.get_news_story(storyId))

## Conclusions

This tutorial covers the following **basic functions** of the `eikon` Python wrapper package for the Eikon Data API:

* `eikon` package
* `ek.get_symbology()`: converting symbology types
* `ek.get_data()`: retrieve historical data
* `ek.get_timeseries()`: retrieve historical time series data
* `ek.get_news_headlines()`: retrieve news headlines
* `ek.get_news_story()`: retrieve full news texts

## Eikon Data API Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Eikon Search Bar.

* [Article on Chains](https://developers.thomsonreuters.com/article/simple-chain-objects-ema-part-1)

<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">